In [16]:
!pip install gdown

In [17]:
import gdown
import torch
import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from scipy.stats import wasserstein_distance
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AffinityPropagation
import matplotlib.pyplot as plt
import random

In [18]:
seed = 1
random.seed(seed)
np.random.seed(seed)

In [19]:
def get_top_values(arr, p):
    # Ensure p is a valid percentage
    if not (0 <= p <= 100):
        raise ValueError("Percentage p must be between 0 and 100.")

    # Calculate the number of elements to retain
    num_elements_to_retain = int(len(arr) * (100 - p) / 100)

    # If num_elements_to_retain is 0, return an empty array
    if num_elements_to_retain == 0:
        return np.array([])

    # Find the indices of the largest num_elements_to_retain elements
    indices_to_retain = np.argpartition(arr, -num_elements_to_retain)[-num_elements_to_retain:]

    # Return only the top (100-p)% of elements
    return arr[indices_to_retain]

In [20]:
DATASET_TYPE = "cifar10"
MODEL_TYPE = "cnn"

DATASET_TYPE=None
NUMBER_OF_CLASSES=10

In [21]:
class Net(nn.Module):
    def __init__(
        self,
    ):
        super(Net, self).__init__()

        print("made model")
        self.conv1 = nn.Conv2d(1, 6, 5) #!
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        


    def forward(self, x):
        out = None

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        out = x
        return out

In [22]:
def array_prune(array, P):


    array = np.array(array)
    n_elements = array.size

    num_to_zero = int(P*len(arr))
    if num_to_zero == 0:
        num_to_zero = 1

    # Get the indices of the smallest num_to_zero elements
    #smallest_indices = np.argpartition(array, num_to_zero - 1)[:num_to_zero]

    # Create a copy and set the smallest elements to zero
    #modified_array = array.copy()
    #modified_array[smallest_indices] = 0

    #return modified_array

    threshold_value = np.percentile(array, P * 100)

    # Set values below or equal to the threshold to 0
    modified_arr = np.where(array <= threshold_value, 0, array)

    return modified_arr

In [23]:
def remove_batchnorm_params(model):
    state_dict = model.state_dict()

    # Create a new dictionary excluding BatchNorm parameters
    filtered_state_dict = {key: value for key, value in state_dict.items() if "running_mean" not in key and "running_var" not in key}

    #print("Removed BatchNorm parameters:")
    #for key in state_dict.keys():
        #if "running_mean" in key or "running_var" in key:
            #print(f"  - {key}")

    return filtered_state_dict

In [24]:
def remove_batchnorm_params(state_dict):
    # Filter out BatchNorm parameters
    filtered_state_dict = {
        key: value
        for key, value in state_dict.items()
        if "running_mean" not in key and "running_var" not in key and "bn" not in key
    }

    # Print removed BatchNorm parameters for verification
    # print("Removed BatchNorm parameters:")
    # for key in state_dict.keys():
    #     if "running_mean" in key or "running_var" in key or "bn" in key:
    #         print(f"  - {key}")

    return filtered_state_dict
def model_parameters_to_numpy(path):
    # Load the model's state dictionary
    model = torch.load(path, map_location=torch.device('cpu'))
    # model.eval()  # Set the model to evaluation mode
    #print("Model loaded successfully.")
    #model = model.state_dict()

    # Initialize an empty list to store parameters
    params = []

    # Iterate through the model to extract parameters
    model = remove_batchnorm_params(model)
    for name, param in model.items():
        if 'bias' not in name:  # Exclude bias terms
            params.append(param.detach().cpu().numpy())  # Convert to numpy array

    # Return the parameters as a numpy array
    return np.array(params, dtype=object)

def load_model_params(model_path):
    state_dic = torch.load(model_path, map_location=torch.device('cpu'))
    
    model = Net()
    model.train()
    model.load_state_dict(state_dic)

    # raw_parameters = []
    # for name, parameter in model.named_parameters():
    #     print(f"{name}")
    #     grads = parameter.grad.abs().view(-1).cpu().numpy()
    # for _, grad in grads:
    #     raw_parameters.append(grad)
    raw_parameters = torch.cat([param.view(-1) for param in model.parameters()]).detach().cpu().numpy()
    return raw_parameters

In [25]:
def top_to_one_other_zero(arr, P):
    flat_arr = arr.flatten()
    threshold_value = np.percentile(flat_arr, P * 100)

    # Create a new array where values below the threshold are 0 and others are 1
    result = np.where(arr <= threshold_value, 0, 1)

    return result

In [26]:
# Example usage
import numpy as np
arr = np.array([0.5, 2.1, 1.8,0.5, 2.1, -3.3, 0.7, 1.8,0.5, 2.1, -3.3, 0.7, 1.8,0.5, 2.1, -3.3, 0.7, 1.8])
P = .4
result = top_to_one_other_zero(arr, P)
result2 = array_prune(arr, P)
len(arr), result, result2,np.count_nonzero(result) ,np.count_nonzero(result2)

(18,
 array([0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1]),
 array([0. , 2.1, 1.8, 0. , 2.1, 0. , 0.7, 1.8, 0. , 2.1, 0. , 0.7, 1.8,
        0. , 2.1, 0. , 0.7, 1.8]),
 11,
 11)

In [27]:
def group_clients_by_cluster(labels):
    clusters = {}
    for client_id, cluster_id in enumerate(labels):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(client_id)
    return clusters

In [28]:
import numpy as np
from collections import defaultdict
def cluster_to_list(arr):
    cluster_dict = defaultdict(list)
    for index, cluster in enumerate(arr):
        cluster_dict[cluster].append(index)
    clusters = [indices for _, indices in sorted(cluster_dict.items())]

    return clusters

## Load From Lab Google Drive

## Cosine

In [29]:
N=10
params=[]
NUMBER_OF_EPOCHS = 480 # values are: 6, 12, 18, 24, 30, 36, 42, 48, 480
DATASET = "svhn"  # values are: cifar10 or svhn

sub_path = None
pretrained = True
new = True


for i in range(N):
    path = f"/home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_{i}_round_2_of_clustering.pth"
    if not os.path.exists(path):
        print(f"File {path} does not exist. Skipping...")
        continue
    try:
        # node = model_parameters_to_numpy(path)
        raw_parameters = load_model_params(path)
        # param = np.concatenate([p.ravel() for p in node])
        params.append(raw_parameters)
        print(f"Successfully loaded: {path}")
    except Exception as e:
        print(f"Error loading {path}: {e}")

made model
Successfully loaded: /home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_0_round_2_of_clustering.pth
made model
Successfully loaded: /home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_1_round_2_of_clustering.pth
made model
Successfully loaded: /home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_2_round_2_of_clustering.pth
made model
Successfully loaded: /home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_3_round_2_of_clustering.pth
made model
Successfully loaded: /home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_4_round_2_of_clustering.pth
made model
Successfully loaded: /home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_5_round_2_of_clustering.pth
made model
Successfully loaded: /home/mmroshani/Documents/lab/SAFE-PFL-HE/open-source/safe-pfl/models/node_6_round_2_of_clustering.pth
made model
Successfully loaded: /home/mmroshani/Documen

/tmp/ipykernel_36975/3739064455.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dic = torch.load(model_path, map_location=torch.device('cpu'))


In [30]:
params[0]

array([ 0.0489625 ,  0.03767579,  0.06528731, ..., -0.18624394,
       -0.22135861, -0.1435833 ], shape=(44426,), dtype=float32)

In [31]:
N = len(params)
print("Number of parameters (N):", N)
Cosine_similarity = np.zeros((N, N))
for i in range(N):
    for j in range(i+1, N):
        Cosine_similarity[i][j] = Cosine_similarity[j][i] = cosine_similarity(params[i].reshape(1, -1), params[j].reshape(1, -1))

Number of parameters (N): 10


/tmp/ipykernel_36975/1528442470.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Cosine_similarity[i][j] = Cosine_similarity[j][i] = cosine_similarity(params[i].reshape(1, -1), params[j].reshape(1, -1))
/tmp/ipykernel_36975/1528442470.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Cosine_similarity[i][j] = Cosine_similarity[j][i] = cosine_similarity(params[i].reshape(1, -1), params[j].reshape(1, -1))
/tmp/ipykernel_36975/1528442470.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. 

In [32]:
# Cosine_distance = 1 - Cosine_similarity
ap = AffinityPropagation(affinity='precomputed', random_state=0).fit(Cosine_similarity)
cosine_labels = ap.labels_
print(Cosine_similarity)
clusters = group_clients_by_cluster(cosine_labels)

print("Clients clustering based on their dataset:", clusters)
print("Cluster labels for each parameter:", cosine_labels)

[[0.         0.99999976 0.99999976 0.99999976 0.99999976 0.54236591
  0.54236591 0.69142139 0.69142139 0.69142139]
 [0.99999976 0.         0.99999976 0.99999976 0.99999976 0.54236591
  0.54236591 0.69142139 0.69142139 0.69142139]
 [0.99999976 0.99999976 0.         0.99999976 0.99999976 0.54236591
  0.54236591 0.69142139 0.69142139 0.69142139]
 [0.99999976 0.99999976 0.99999976 0.         0.99999976 0.54236591
  0.54236591 0.69142139 0.69142139 0.69142139]
 [0.99999976 0.99999976 0.99999976 0.99999976 0.         0.54236591
  0.54236591 0.69142139 0.69142139 0.69142139]
 [0.54236591 0.54236591 0.54236591 0.54236591 0.54236591 0.
  1.         0.6161077  0.6161077  0.6161077 ]
 [0.54236591 0.54236591 0.54236591 0.54236591 0.54236591 1.
  0.         0.6161077  0.6161077  0.6161077 ]
 [0.69142139 0.69142139 0.69142139 0.69142139 0.69142139 0.6161077
  0.6161077  0.         0.99999994 0.99999994]
 [0.69142139 0.69142139 0.69142139 0.69142139 0.69142139 0.6161077
  0.6161077  0.99999994 0.    

**Coordinate-Based**

#! read from all_model_parameters_ordered_by_importance_for_client_*.csv and cluster


In [33]:
P=0.8
coordinates=[]
for i in range(N):
  tops=top_to_one_other_zero(params[i], P)
  coordinates.append(tops)
ID_similarity=np.zeros((N,N))
for i in range(N):
  for j in range(i+1,N):
    ID_similarity[i][j]=cosine_similarity(coordinates[i],coordinates[j])

ID_distance = -ID_similarity
ap = AffinityPropagation(affinity="precomputed", random_state=0).fit(ID_distance)
ID_labels = ap.labels_
ID_similarity_show = np.round(ID_similarity, 2)


ValueError: Expected 2D array, got 1D array instead:
array=[1. 1. 1. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
Cosine_similarity_show=np.round(Cosine_similarity, 2)
print(cosine_labels)
Cosine_similarity_show

In [ ]:
ID_similarity_show=np.round(ID_similarity, 2)

print(ID_labels)
ID_similarity_show

In [ ]:
matrix1=Cosine_similarity.flatten()
matrix2=ID_similarity.flatten()
from scipy.stats import pearsonr, spearmanr

corr_pearson, _ = pearsonr(matrix1, matrix2)
corr_spearman, _ = spearmanr(matrix1, matrix2)
mae = np.mean(np.abs(matrix1 - matrix2))
diff=matrix1- matrix2
min_,max_,avg_=np.min(diff),np.max(diff),np.average(diff)
corr_pearson, corr_spearman, mae, min_,max_,avg_

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def find_regression_coefficients(x, y):
    x = np.array(x).reshape(-1, 1)
    y = np.array(y)

    # Create and fit the regression model
    model = LinearRegression()
    model.fit(x, y)

    return model.coef_[0], model.intercept_
slope, intercept = find_regression_coefficients(matrix1, matrix2)
cofficient=f"cosine(x,y)={round(slope,3)} ID_sim(x,y) {round(intercept,3)}"
cofficient

In [ ]:
# Filter non-zero values for both matrices
non_zero_indices_x = np.nonzero(matrix1)
non_zero_indices_y = np.nonzero(matrix2)

# Get non-zero values
x_non_zero = matrix1[non_zero_indices_x]
y_non_zero = matrix2[non_zero_indices_y]

# Create scatter plot for non-zero values
plt.figure(figsize=(8, 6))

# Plot X values (Cosine Similarity) in blue
plt.scatter(range(len(x_non_zero)), x_non_zero, color='blue', alpha=0.7, label='Cosine Similarity (X)')

# Plot Y values (Approximation Similarity) in red
plt.scatter(range(len(y_non_zero)), y_non_zero, color='red', alpha=0.7, label='Approximation Similarity (Y)')

# Add labels and title
plt.xlabel('Data Index', fontsize=12)
plt.ylabel('Similarity Values', fontsize=12)
plt.title(f"Pearson Correlation: {round(corr_pearson,3)} Spearman Correlation: {round(corr_spearman,3)} "+cofficient, fontsize=14)
plt.legend()
plt.grid(alpha=0.4)

plt.show()





## **Clustering and Silouhette**

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import pairwise_distances


data = np.array([
    [4948, 0,5293, 0, 0,0,0,0,0,0,],
[0,3466, 0, 0, 0,0,0,0,0, 2330,],
[0,3465,5292, 0, 0,0,0,0,0,0,],
[0, 0, 0,4249,3729,0,0,0,0,0,],
[0,3465, 0, 0,3729,0,0,0,0,0,],
[0, 0, 0, 0, 0, 6882,0, 1865,0,0,],
[0, 0, 0, 0, 0,0, 2864, 1865,0,0,],
[0, 0, 0, 0, 0,0, 2863, 1865,0,0,],
[0, 0, 0,4248, 0,0,0,0, 5045,0,],
[0,3465, 0, 0, 0,0,0,0,0, 2329,]
])

# Calculate cosine similarity
cosine_dist_opt = pairwise_distances(data, metric='cosine')
cosine_similarities_opt = -cosine_dist_opt  # Convert distances to negative similarities for AP

# Determine the preference based on the cosine similarity matrix
preference = np.median(cosine_similarities_opt)

# Affinity Propagation with cosine similarity
clustering = AffinityPropagation(affinity='precomputed', preference=preference, random_state=0).fit(cosine_similarities_opt)

cluster_labels = clustering.labels_
# Calculate the silhouette score


silhouette_avg_opt = silhouette_score(data, cluster_labels, metric='cosine')
silhouette_avg_cosine = silhouette_score(data, cosine_labels, metric='cosine')
silhouette_avg_method = silhouette_score(data, ID_labels, metric='cosine')


# Organizing the final clusters as [[], [], ...]
num_clusters = len(np.unique(cluster_labels))
clusters = [[] for _ in range(num_clusters)]
for index, label in enumerate(cluster_labels):
    clusters[label].append(index)

# Output the clusters and silhouette score
print(f"Silhouette Score with[{clusters}]: {silhouette_avg_opt}")
print(f"Silhouette Score_cosine with [{cluster_to_list(cosine_labels)}]: {silhouette_avg_cosine}")
print(f"Silhouette Score_ID method with [{cluster_to_list(ID_labels)}]: {silhouette_avg_method}")


## Others

In [ ]:
similarity_vector=[]
for p in range (0,100):
    similarity_vector.append(cosine_similarity(params[1], array_prune(params[1], p/100)))

In [ ]:
similarity_vector_ID=[]
for p in range (1,100):
    similarity_vector_ID.append(cosine_similarity(params[1], top_to_one_other_zero(params[1], p/100)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
values = np.random.rand(100)
plt.figure(figsize=(10, 5))  # Optional: Adjust the figure size
plt.plot(similarity_vector, marker='o', linestyle='-', label='similarity of prun')
plt.plot(similarity_vector_ID, marker='o', linestyle='-', label='similarity of ID')
plt.title('Plot of 100 Values')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

## Euclidean

In [ ]:
Eu_similarity=np.zeros((N,N))
for i in range(N):
    for j in range(i + 1, N):
        # Calculate Euclidean distance
        Eu_similarity[i][j] =Eu_similarity[j][i] = np.linalg.norm(params[i] - params[j])

Eu_distance = -Eu_similarity
ap = AffinityPropagation(affinity="precomputed").fit(Eu_distance)
labels = ap.labels_

print("Cluster labels for each prune:", labels)

## Wasserstein (!!!Takes too long!!!!)

In [ ]:
W_similarity=np.zeros((N,N))
for i in range(N):
  for j in range(i + 1, N):
    W_similarity[i][j] =W_similarity[j][i] = wasserstein_distance(params[i], params[j])

W_distance=W_similarity
ap = AffinityPropagation(affinity="precomputed").fit(W_distance)
labels = ap.labels_

print("Cluster labels for each prune:", labels)

In [ ]:
a=params[0]
b=params[1]
c=params[2]

x=np.linalg.norm(b)
x

In [ ]:
y=np.linalg.norm(c)
y

In [ ]:
z=np.dot(b,c)
z

In [ ]:
print(z,x,y,x*y,z/(x*y))

In [ ]:
z/x*y

In [ ]:
# b=params[0]
# #a_=prun_0
# #b_=prun_1
# eps_a=np.linalg.norm(a)-np.linalg.norm(a_)
# eps_b=np.linalg.norm(b)-np.linalg.norm(b_)
# print("epsilons:", eps_a, eps_b)
# if eps_a<=np.linalg.norm(a-a_):
#   print("satisfy")

# if eps_b<=np.linalg.norm(b - b_):
#   print("satisfy")
# delta=cosine_similarity(a,b)-cosine_similarity(a_,b_)
# print(delta)

# check_delta=eps_a/np.linalg.norm(a)+eps_b/np.linalg.norm(b)+(eps_a*eps_b)/np.linalg.norm(a)*np.linalg.norm(b)
# print(check_delta)

